In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Download dataset from HF
----------------

In [ ]:
!git lfs install
!git clone https://huggingface.co/datasets/nlp-guild/Nonlinear-System-Identification-with-Deep-Learning

def load_dataset(path='dataset.npy'):
    """
    :return:
        f_and_xs: numpy array of size [sample_number, channels, sample_length]
        label_0, label_1, label_2: one-hot encodes of size [sample_number, number_bins]
    """

    r = np.load(path, allow_pickle=True).item()
    f_and_xs = r['f_and_xs']
    label_0 = r['l_0']
    label_1 = r['l_1']
    label_2 = r['l_2']
    return f_and_xs, label_0, label_1, label_2

f_and_xs, label_0, label_1, label_2 = load_dataset('/content/Nonlinear-System-Identification-with-Deep-Learning/dataset.npy')

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'Nonlinear-System-Identification-with-Deep-Learning'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 39 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.
tcmalloc: large alloc 1471086592 bytes == 0x560c54306000 @  0x7f26807202a4 0x560c16f9578f 0x560c16f728db 0x560c16f275b3 0x560c16ecb34a 0x560c16ecb806 0x560c16ee8ad1 0x560c16ee9069 0x560c16ee9593 0x560c16f8e482 0x560c16e2ecc2 0x560c16e15a75 0x560c16e16735 0x560c16e1573a 0x7f267fa67c87 0x560c16e1578a


Create dataloader
----------------

In [ ]:
f_and_x = np.zeros((100,))
our_data = []
n_sample = 500
for i in range(2500):
  f = f_and_xs[i, 0, :n_sample].astype('float32')
  x1 = f_and_xs[i, 1, :n_sample].astype('float32')
  x2 = f_and_xs[i, 2, :n_sample].astype('float32')
  x3 = f_and_xs[i, 3, :n_sample].astype('float32')
  x4 = f_and_xs[i, 4, :n_sample].astype('float32')
  x5 = f_and_xs[i, 5, :n_sample].astype('float32')
  x6 = f_and_xs[i, 6, :n_sample].astype('float32')
  input = np.hstack([f, x1, x2, x3, x4, x5, x6]).astype('float32')
  # label = np.hstack([k1_labels[i], k2_labels[i], alpha_labels[i]]).astype('float32')
  label = label_0[i].astype('float32')
  our_data.append((input, label))

# random.shuffle(our_data)
BATCH_SIZE = 96
shuffle = True
our_train_iterator = data.DataLoader(our_data[:1920], shuffle=shuffle, batch_size=BATCH_SIZE)
# our_valid_iterator = data.DataLoader(our_data[1920:], shuffle=shuffle, batch_size=BATCH_SIZE)
our_valid_iterator = data.DataLoader(our_data[:1920], shuffle=shuffle, batch_size=BATCH_SIZE)
# our_test_iterator = data.DataLoader(our_data, shuffle=shuffle, batch_size=BATCH_SIZE)

input_dim = np.shape(input)[0]
output_dim = np.shape(label)[0]

In [ ]:
class MyLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, y):
        return -torch.mean(y*torch.log(x) + (1-y) * torch.log(1-x))

def accuracy(output, target, k):
  top_k_index = torch.topk(output, k).indices
  # print(top_k_index)
  goal = torch.argmax(target, axis=1)
  # print(goal)
  hit = [1 if goal[i] in top_k_index[i] else 0 for i in range(target.size(0))]
  # print(hit)
  return np.mean(hit)

def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred = model(x)
        # print(y_pred)

        loss = criterion(y_pred, y)

        acc = accuracy(y_pred, y, 1)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = accuracy(y_pred, y, 1)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Model
----------------

In [ ]:
from re import A

class Reshape(nn.Module):
  def __init__(self, s): 
    super(Reshape, self).__init__() 
    self.shape = s 

  def forward(self, x):
    return x.view(self.shape)

track_running_stats = True
class ourMLP(nn.Module):
    def __init__(self, input_dim, output_dim, middle_channels, BN=True, track_running_stats=True):
        super().__init__()
        self.net = nn.Sequential()

        for i in range(len(middle_channels)):
          if i == 0:
            self.net.add_module(f"FC{i}", nn.Linear(input_dim, middle_channels[0]))
            if BN:
              self.net.add_module(f"BN{i}", torch.nn.BatchNorm1d(middle_channels[0], eps=1e-05, momentum=0.1, affine=True, track_running_stats=track_running_stats))
            self.net.add_module(f"relu{i}", nn.ReLU())
          else:
            self.net.add_module(f"FC{i}", nn.Linear(middle_channels[i-1], middle_channels[i]))
            if BN:
              self.net.add_module(f"BN{i}", torch.nn.BatchNorm1d(middle_channels[i], eps=1e-05, momentum=0.1, affine=True, track_running_stats=track_running_stats))
            self.net.add_module(f"relu{i}", nn.ReLU())

        self.net.add_module("last_layer", nn.Linear(middle_channels[-1], output_dim))

    def forward(self, x):

        x = self.net(x)
        # print(x.shape)

        x = F.softmax(x)
        return x

model = ourMLP(input_dim, output_dim, [1000, 1000, 500, 500], BN=True, track_running_stats=True)
print(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
total = sum([param.nelement() for param in model.parameters()])
print("Number of parameter: %.2fM" % (total/1e6))

criterion = MyLoss()
model = model.to(device)
criterion = criterion.to(device)

3500
Number of parameter: 5.26M


Training
----------------

In [ ]:
EPOCHS = 100

best_valid_loss = float('inf')
# model = ourMLP(INPUT_DIM, OUTPUT_DIM)
# model = model.to(device)
for epoch in trange(EPOCHS):

    start_time = time.monotonic()

    # train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    # valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    train_loss, train_acc = train(model, our_train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, our_valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    end_time = time.monotonic()

    # epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    # print(f'\tTrain Loss: {train_loss:.3f}')
    # print(f'\t Val. Loss: {valid_loss:.3f}')
    print(f"####################### epoch {epoch} #######################")

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

	Train Loss: 0.158 | Train Acc: 63.39%
	 Val. Loss: nan |  Val. Acc: 15.52%
####################### epoch 0 #######################


...

Training:   0%|          | 0/20 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

	Train Loss: 0.098 | Train Acc: 77.19%
	 Val. Loss: 0.140 |  Val. Acc: 78.54%
####################### epoch 4 #######################


Training:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

Inference
----------------

In [ ]:
model.eval()
index = 13
x, label = our_data[index]
x = np.reshape(x, (1, 3500))
print(label)
p = model(torch.from_numpy(x).to(device))
print(p[0])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
tensor([5.3803e-04, 2.7432e-03, 2.3292e-01, 7.2581e-01, 3.5428e-02, 1.1741e-03,
        3.2633e-04, 3.3226e-04, 3.2770e-04, 3.9935e-04],
       grad_fn=<SelectBackward0>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
